In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
# Basic use of PyTorch
# 1. Manually create the parameter tensors
# 2. Compute the forward loss
# 3. Use auto differentiation to compute gradients
# 4. Update parameters
x = torch.tensor([[0.0, 0.0],
                  [0.0, 1.0],
                  [1.0, 0.0],
                  [1.0, 1.0]])
y = torch.tensor([[0.0],
                  [1.0],
                  [1.0],
                  [0.0]])

n = x.shape[0]
p = x.shape[1]
d = y.shape[1]
r = 10

np.random.seed(123)
torch.random.manual_seed(123)

w1 = torch.randn(p, r, requires_grad=True)
b1 = torch.randn(r, 1, requires_grad=True)
w2 = torch.randn(r, d, requires_grad=True)
b2 = torch.randn(d, 1, requires_grad=True)

nepoch = 1000
learning_rate = 0.1

for i in range(nepoch):
    z1 = x.matmul(w1) + b1.transpose(0, 1)
    a1 = F.softplus(z1)
    z2 = a1.matmul(w2) + b2.transpose(0, 1)
    phat = torch.sigmoid(z2)

    loss = torch.mean(-y * torch.log(phat) - (1.0 - y) * torch.log(1.0 - phat))
    loss.backward()

    if i % 50 == 0:
        print(f"iteration {i}, loss = {loss.item()}, prediction = {phat.detach().numpy().squeeze()}")

    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        b1 -= learning_rate * b1.grad
        w2 -= learning_rate * w2.grad
        b2 -= learning_rate * b2.grad
        
        w1.grad = None
        b1.grad = None
        w2.grad = None
        b2.grad = None

In [ ]:
# Basic use of Modules
# 1. Define a class that encapsulates the parameters and the forward pass
# 2. Define an optimizer
# 3. Compute the loss function value
# 4. Use auto differentiation to compute gradients
# 5. Update parameters
x = torch.tensor([[0.0, 0.0],
                  [0.0, 1.0],
                  [1.0, 0.0],
                  [1.0, 1.0]])
y = torch.tensor([[0.0],
                  [1.0],
                  [1.0],
                  [0.0]])

n = x.shape[0]
p = x.shape[1]
d = y.shape[1]
r = 10

class MyModel(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        z1 = self.fc1(x)
        a1 = F.softplus(z1)
        z2 = self.fc2(a1)
        phat = torch.sigmoid(z2)
        return phat

np.random.seed(123)
torch.random.manual_seed(123)

model = MyModel(in_dim=p, out_dim=d, hidden_dim=r)
print(model, "\n")
print(list(model.parameters()))

In [ ]:
nepoch = 2000
learning_rate = 0.1
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)

for i in range(nepoch):
    phat = model(x)

    loss = torch.mean(-y * torch.log(phat) - (1.0 - y) * torch.log(1.0 - phat))

    if i % 100 == 0:
        print(f"iteration {i}, loss = {loss.item()}, prediction = {phat.detach().numpy().squeeze()}")

    opt.zero_grad()
    loss.backward()
    opt.step()